# References
[Tutorial](https://gilberttanner.com/blog/tensorflow-object-detection-with-tensorflow-2-creating-a-custom-model/) <br>
[Official docs](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html#general-remarks)

# Folder structures 
```
<your-repo>
|_ annotated
    |_ label_map.pbtxt
    |_ 1.xml
    |_ 2.xml
    |_ ...xml
|_ images
    |_ 1.jpg
    |_ 2.jpg
    |_ ...jpg
|_ models (should be from TF2 repo)
    |_ <custom-model>
        |_pipeline.config (modified config)
|_ records
    |_ train.record
    |_ test.record
|_ csv
    |_ train_annot.csv
    |_ test_annot.csv
|_ main.ipynb
```

# Setup

# Installation
Only need to run these codeblocks once.

In [7]:
# clone tensorflow models
if not path.exists('models'):
    !git clone https://github.com/tensorflow/models.git

### Protoc should be downloaded beforehand from [google repo](https://github.com/protocolbuffers/protobuf/releases) search for v3.4.0 and place it under models/research directory.
v3.5 and above have deprecated the following command 
<br>`!protoc object_detection/protos/*.proto --python_out=.`. 

In [17]:
%cd models/research/
# runs protoc 
!protoc object_detection/protos/*.proto --python_out=.

# copy setup.py to models/research, do this manually if the code does not run
# !cp object_detection/packages/tf2/setup.py .

# install object_detection API
!python -m pip install .

%cd ../../

D:\Repository\tracking-vision\models\research
D:\Repository\tracking-vision


### There should be no errors if installation is correct

In [2]:
!python ./models/research/object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.9.12: C:\Users\shade\miniconda3\envs\tensorflow\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-06-12 18:34:37.466397: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-12 18:34:38.061330: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2780 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
C:\Users\shade\miniconda3\envs\tensorflow\lib\site-packages\object_detection\builders\model_builder.py:1102: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(('Building experimental DeepMAC meta-arc

# Preprocess data

In [27]:
import os 
from os import path
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import wget
from sklearn.model_selection import train_test_split


IMG_DIR = "./images/raw"
SAVE_DIR = "./images/processed"
XML_DIR = "./annotation/xml"
XML_RAW_DIR = "./annotation/raw_xml"

if not path.exists('dataset'):
  os.mkdir('dataset')

In [13]:
# optional pre process image
from PIL import Image
import matplotlib.pyplot as plt
import cv2

def crop_center(pil_img, frac):
    left = img.size[0]*((1-frac)/2)
    upper = img.size[1]*((1-frac)/2)
    right = img.size[0]-((1-frac)/2)*img.size[0]
    bottom = img.size[1]-((1-frac)/2)*img.size[1]
    
    return img.crop((left, upper, right, bottom))

def crop_center_fixed(img, size):
    width, height = img.size

    left = (width - size)/2
    top = (height - size)/2
    right = (width + size)/2
    bottom = (height + size)/2
    
    return img.crop((left, top, right, bottom))

def preprocess_cv2():
    for i, path in enumerate(os.listdir(IMG_DIR)):
        img = cv2.imread(os.path.join(IMG_DIR, path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # reduce height
        if img.shape[0] > 640:
            h = int(img.shape[0] * 0.15)
            img = img[h:h+640]

        # cut into multiple columns
        for c in range(0, img.shape[1], 640):
            img_name =  str(i+1) + '_image_' + str(c) + '.jpg'
            img_final = img[0:640, c:c+640,:]
            cv2.imwrite(os.path.join(SAVE_DIR, img_name), img_final)

        if (i+1)%10 == 0: 
            plt.figure(figsize=(8,6))
            plt.imshow(img)
        if i+1 == 1: break
            
def preprocess_pil():
    size = (1280, 640)
    left = (0, 0, 640, 640)
    right = (640, 0, 1280, 640)
    crops = {'left': left, 'right': right}
    for i, path in enumerate(os.listdir(IMG_DIR)):
        img = Image.open(os.path.join(IMG_DIR, path))
        img = img.convert('RGB')
        img = img.resize(size)
        
        for k, v in crops.items():
            proc = img.crop(v)
            proc.save(f"{SAVE_DIR}/{path.split('.')[0]}_{k}.jpg")
        
    return 'processed ' + str(i+1) + ' images.'

preprocess_pil()

'processed 84 images.'

In [28]:
# create label_img.pbtxt
def create_label_img(classes):    
    with open('./annotation/label_map.pbtxt', 'w') as f:
        for i, val in enumerate(classes):
            append = "" if i==0 else "\n"
            item = append + "item {\n\tname: \"" + val + "\" \n\tid: " + str(i+1) +"\n}"
            f.write(item)
    print('Created label map for ' + str(classes))

In [29]:
# turn .xml files into csv
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                root.find('filename').text,
                int(root.find('size')[0].text),
                int(root.find('size')[1].text),
                member[0].text,
                int(member[4][0].text),
                int(member[4][1].text),
                int(member[4][2].text),
                int(member[4][3].text)
            )
            xml_list.append(value)
    col_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=col_name)
    return xml_df

xml_df = xml_to_csv(XML_DIR)

train_data, test_data = train_test_split(xml_df, test_size=0.2, random_state=69)

In [30]:
xml_df.value_counts('class')

class
melantha    18
steward     16
beagle      14
hibiscus    14
ansel       13
lava        13
popukar     13
spot        13
plume       12
vanilla     12
kroos       11
midnight    11
cardigan    10
fang        10
catapult     7
orchid       7
dtype: int64

In [31]:
train_data.value_counts('class')

class
hibiscus    13
melantha    13
beagle      12
steward     12
ansel       11
lava        11
popukar     11
spot        11
cardigan     9
midnight     9
vanilla      9
plume        8
fang         7
kroos        7
catapult     6
orchid       6
dtype: int64

In [32]:
# downsample majority classes
from sklearn.utils import resample

class_counts = train_data.value_counts('class')
class_sum = 0
class_avg = 0
for count in class_counts:
    class_sum = class_sum + count
    
class_avg = int(class_sum/len(class_counts))

train_majority = train_data.groupby('class').filter(lambda x: len(x) > class_avg)
train_minority = train_data.groupby('class').filter(lambda x: len(x) <= class_avg)

def resampling(df, avg):
    temp = pd.DataFrame()
    for classes in df['class'].unique():
        data = df[df['class'] == classes]
        resampled = resample(data, n_samples=avg, random_state=69)
        temp = pd.concat([temp, resampled])
    return temp
    
downsampled = resampling(train_majority, class_avg)
upsampled = resampling(train_minority, class_avg)

train_data = pd.DataFrame()
train_data = pd.concat([downsampled, upsampled])
train_data.value_counts('class')

class
ansel       9
beagle      9
cardigan    9
catapult    9
fang        9
hibiscus    9
kroos       9
lava        9
melantha    9
midnight    9
orchid      9
plume       9
popukar     9
spot        9
steward     9
vanilla     9
dtype: int64

In [33]:
test_data.value_counts('class')

class
melantha    5
kroos       4
plume       4
steward     4
fang        3
vanilla     3
ansel       2
beagle      2
lava        2
midnight    2
popukar     2
spot        2
cardigan    1
catapult    1
hibiscus    1
orchid      1
dtype: int64

In [34]:
train_data.to_csv('dataset/train_annot.csv', index=None)
test_data.to_csv('dataset/test_annot.csv', index=None)

create_label_img(xml_df['class'].unique())
print(xml_df['class'].nunique())
print("Train data: {}".format(train_data['class'].nunique()))
print("Test data: {}".format(test_data['class'].nunique()))
print(train_data.head())
print(test_data.head())

Created label map for ['hibiscus' 'popukar' 'melantha' 'lava' 'midnight' 'spot' 'cardigan'
 'beagle' 'steward' 'plume' 'orchid' 'ansel' 'vanilla' 'fang' 'kroos'
 'catapult']
16
Train data: 16
Test data: 16
                    filename  width  height  class  xmin  ymin  xmax  ymax
164   1651165472990_left.jpg    640     640  ansel   151   194   273   324
152   1651165401052_left.jpg    640     640  ansel   149   201   272   314
107   1629312056176_left.jpg    640     640  ansel   455   194   545   326
82   1629306745515_right.jpg    640     640  ansel   490   188   611   330
17   1629306252581_right.jpg    640     640  ansel   321   188   424   340
                    filename  width  height    class  xmin  ymin  xmax  ymax
182  1651165551493_right.jpg    640     640  steward   107   174   210   311
97   1629308257499_right.jpg    640     640     fang   104   195   211   323
172  1651165482319_right.jpg    640     640  vanilla   206   178   308   340
92    1629308110407_left.jpg    640 

In [35]:
!python generate_tf_records.py \
    -l ./annotation/label_map.pbtxt -o ./dataset/train.record -i ./images/processed -csv ./dataset/train_annot.csv

!python generate_tf_records.py \
    -l ./annotation/label_map.pbtxt -o ./dataset/test.record -i ./images/processed -csv ./dataset/test_annot.csv

INFO:Successfully created the TFRecords: ./dataset/train.record
INFO:Successfully created the TFRecords: ./dataset/test.record


In [3]:
# verify tensor records
import tensorflow as tf 
raw_dataset = tf.data.TFRecordDataset("records/train.record")

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

# Config

In [69]:
num_classes = xml_df['class'].nunique()
batch_size = 4
# num_steps = 5000
num_eval_steps = 1000

train_record_path = 'records/train.record'
test_record_path = 'records/test.record'
model_dir = 'models/my_ssd_resnet50_v1_fpn'
labelmap_path = 'annotated/label_map.pbtxt'

pipeline_config_path = 'models/my_ssd_resnet50_v1_fpn/pipeline.config'
fine_tune_checkpoint = 'models_pretrained/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0'

In [70]:
import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:
    # Set labelmap path
    config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)

    # Set fine_tune_checkpoint path
    config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)

    # Set train tf-record file path
#     config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
#                   'input_path: "{}"'.format(train_record_path), config)

    # Set test tf-record file path
#     config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
#                   'input_path: "{}"'.format(test_record_path), config)

    # Set number of classes.
    config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)

    # Set batch size
    config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)

#     Set training steps
#     config = re.sub('num_steps: [0-9]+',
#                   'num_steps: {}'.format(num_steps), config)

    f.write(config)

# Training

```
python model_main_tf2.py --model_dir=models/<model-dir> --pipeline_config_path=models/<model-dir>/pipeline.config

```

In [ ]:
python model_main_tf2.py --model_dir=models/my_model --pipeline_config_path=models/my_model/pipeline.config --alsologtostderr
python model_main_tf2.py --model_dir=models/my_model_v2 --pipeline_config_path=models/my_model_v2/pipeline.config --alsologtostderr --num_train_steps=3000
python model_main_tf2.py --model_dir=models/my_model_lite --pipeline_config_path=models/my_model_lite/pipeline.config --alsologtostderr --num_train_steps=3000
python model_main_tf2.py --model_dir=models/my_model_lite --pipeline_config_path=models/my_model_lite/pipeline.config --checkpoint_dir=models/my_model_lite

In [ ]:
# mobilenet
python model_main_tf2.py --model_dir=out_model/mobilenet --pipeline_config_path=out_model/mobilenet/pipeline.config --alsologtostderr --num_train_steps=3000
python model_main_tf2.py --model_dir=out_model/mobilenet --pipeline_config_path=out_model/mobilenet/pipeline.config --checkpoint_dir=out_model/mobilenet

In [ ]:
# Export model
python .\exporter_main_v2.py --input_type image_tensor --pipeline_config_path .\models\my_model_lite\pipeline.config --trained_checkpoint_dir .\models\my_model_lite\ --output_directory .\exported-models\ak_model_lite

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}